__tf.data__ 模块包含了一些类的类可以用来载入数据，处理数据，并且将其管道到（pipe into）你的model。这个文档通过下面两个例子介绍这个API：
* 从numpy array读取内存中的数据(in-memory data)
* 从csv file中读取行(lines)

## Basic input
从一个array中获取片段是__tf.data__的最简单的方式。  
下面示例的__train_input_fn__ 函数，将数据传递到Estimator

In [1]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Build the Iterator, and return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()

### Argument
这个函数有三个参数。如果参数接收的是"array",那么可以接收能够被__numpy.array__转换为array的任意数据。其中的一个例外是元组（tuple），因为它代表了Datasets中其他的特别的含义。
* __features:__ 字典（或者DATa Frame）{'feature_name':array}包含原始的输入特征。
* __labels:__ array，包含每个示例的label
* __batch_size:__ integer，想要的batch size的值  

在Iris的例子当中，我们用 iris_data.load_data()函数来获得数据，示例如下：

In [ ]:
import iris_data

# Fetch the data
train,test = iris_data.load_data()
features, labels = train
batch_size=100
iris_data.train_input_fn(features, labels, batch_size)

解析train_input_fn()函数
#### Slices
最简单的情况， __tf.data.Dataset.from_tensor_slices__ 函数输入一个array返回__tf.data.Dataset__代表这个array的slices(数组片）。例如，包含 mnist training data的array的形状是（60000，28，28）。将这个array传递到 __from_tensor_slices__ 会返回一个 Dataset对象包含60000个切片，每一个都是28x28的图片，code示例如下1：。
这个代码会打印如下一行，表明在这个dataset中的items的shapes和types。注意，这个dataset并不知道它自己包含了多少个items。
>  \<TensorSliceDataset shapes: (28,28), types: tf.uint8>

上面这个dataset表明一个简单的数组的集合，但是dataset有更多的其他功能。Dataset可以透明的处理任何的嵌套组合的字典或者元组。例如，在features是一个标准字典的情况下，你可以将这个字典的数组(dictionary of arrays)转换成一个字典的数据集(Dataset of dictionarys),代码实现示例如下2：

这里我们看到当一个Dataset包含了结构化的元素，那么Dataset的shapes和types和Dataset有同样的结构。这个打塔塞特种包含了标量字典(dictionaries of scalars),所有的数据类型都是__tf.float64__。

train_input_fn的第一行使用了同样的功能，但是增加了另外一个级别的结构。它创建了一个包含成对的(feature,labels)的dataset。  
余下的代码3：表示label是类型是int64的标量。

In [6]:
# 1:
import tensorflow as tf
train,test = tf.keras.datasets.mnist.load_data()
mnist_x,mnist_y = train

mnist_ds = tf.data.Dataset.from_tensor_slices(mnist_x)
print(mnist_ds)

11468800/11490434 [============================>.] - ETA: 0s<TensorSliceDataset shapes: (28, 28), types: tf.uint8>


In [ ]:
# 2:
dataset = tf.data.Dataset.from_tensor_slices(dict(features))
print(dataset)

## print show 
'''<TensorSliceDataset

  shapes: {
    SepalLength: (), PetalWidth: (),
    PetalLength: (), SepalWidth: ()},

  types: {
      SepalLength: tf.float64, PetalWidth: tf.float64,
      PetalLength: tf.float64, SepalWidth: tf.float64}
>'''

# 3:
# Convert the inputs to a Dataset.
dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
print(dataset)

# print show
'''<TensorSliceDataset
    shapes: (
        {
          SepalLength: (), PetalWidth: (),
          PetalLength: (), SepalWidth: ()},
        ()),

    types: (
        {
          SepalLength: tf.float64, PetalWidth: tf.float64,
          PetalLength: tf.float64, SepalWidth: tf.float64},
        tf.int64)>'''

### Manipulation
现在，Dataset会通过固定的顺序将所有的数据迭代一次，并且每次只产生一个元素。在training之前还需要更多的处理。幸运的儿视， __tf.Dataset__ 类提供一些方法来处理数据用来training。在input函数当中的第二句就有这些方法的优势。
> \# Shuffle, repeat, and batch the examples.  
dataset = dataset.shuffle(1000).repeat().batch(batch_size)

__shuffle__ 方法用一个固定尺寸的缓存在传递这些items的时候将他们洗牌。设定一个大于Dataset例子数目的__buffer_size__可以保证data是被完全乱序的(shuffled).Iris data set只包含有150个例子。

__repeat__ 方法在Dataset达到最后的时候重新启动（restart when it reaches the end)。为了显示epochs的数目，设定 __count__ 参数。

__batch__ 方法收集一些例子并且将他们堆在一起， 来创造batches。 这回在数据shape的基础上增加一个维度。 这个新增的维度是加在第一维的。 下面的代码在MNIST dataset上使用__batch__方法。 这样会让Dataset包含一个3D的数据表达(28,28)图片的数据堆，如下例1：

注意到这个dataset有一个位置的batch size，因为最后一个batch有比较少的元素。

在__train_iput_fn__中，在batching之后，Dataset会拥1D的矢量元素，再次之前这些数据是标量类型的。(Iris data)，如下例2：

In [ ]:
# 1:
print(minst_ds.batch(100))

# print show
'''<BatchDataset
  shapes: (?, 28, 28),
  types: tf.uint8>'''
    
# 2:
print(dataset)
'''<TensorSliceDataset
    shapes: (
        {
          SepalLength: (?,), PetalWidth: (?,),
          PetalLength: (?,), SepalWidth: (?,)},
        (?,)),

    types: (
        {
          SepalLength: tf.float64, PetalWidth: tf.float64,
          PetalLength: tf.float64, SepalWidth: tf.float64},
        tf.int64)>'''

### Return
Estimator的每个__train,evaluate,predict__方法要求输入方程返回一个成对的(features,label)包含tensorflow tensors。 __train_input_fn__ 使用如下一行将Dataset转换为想要的格式。

这个返回的就是一个结构化的TensorFlow tensor，在Dataset中匹配items的布局。 在[Introduction](https://www.tensorflow.org/programmers_guide/low_level_intro?hl=zh-cn)查看有些什么对象，他们怎么布局。

In [ ]:
# Build the Iterator, and return the read end of the pipeline.
features_result, lablels_result = dataset.make_one_shot_iterator().get_next()

print((feature,result,labels_result))

## print show
'''({
    'SepalLength': <tf.Tensor 'IteratorGetNext:2' shape=(?,) dtype=float64>,
    'PetalWidth': <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=float64>,
    'PetalLength': <tf.Tensor 'IteratorGetNext:0' shape=(?,) dtype=float64>,
    'SepalWidth': <tf.Tensor 'IteratorGetNext:3' shape=(?,) dtype=float64>},
Tensor("IteratorGetNext_1:4", shape=(?,), dtype=int64))'''

## Reading a CSV File
在实际中使用最多的Dataset类将数据流从文件传到磁盘中。__tf.data__模块包含了很大一部分的文件阅读器(readers)。下面查看怎么样用Dataset将Iris数据从csv文件怎样解析出来。

下面的例子中调用__iris_data.maybe_download__函数当其必要的时候下载数据，并且返回最后文件的路径。

__iris_data.csv_input_fn__ 函数包可以用Dataset来解析csv文件含了可替换的实现

下面介绍怎么样创建一个可以从本地文件中读取数据的Estimator兼容的输入方程。

In [ ]:
import iris_data
train_path, test_path = iris_data.maybe_download()

### Build the Dataset
首先创建一个__TextLineDataset__对象来每次从文件中读取一样，然后调用__skip__方法跳过文件的第一行(包含的是文件头(header)而不是一个例子):

In [ ]:
ds = tf.data.TextLineDataset(train_path).skip(1)

### Build a csv line parser
最终我们需要解释dataset中的每一行，来产生必要的(features,labels)对。

首先我们建造一个可以解释单独一行的方程。

__iris_data.parse_line__ 通过使用 __tf.decode_csv__ 函数完成了这个任务。

由于Estimators要求features用一个字典来表示，所以我们依赖于Python的内建（built-in）的__dict__和__zip__函数来构建字典。 特征名称是字典的keys。调用字典的__pop__方法从特征字典中移除label领域(filed):

In [8]:
# Metadata describing the text columns
COLUMNS =['SepalLength', 'SepalWidth',
           'PetalLength', 'PetalWidth',
           'label']
FIELD_DEFAULTS = [[0.0], [0.0], [0.0], [0.0], [0]]
def parse_line(line):
    # Decode the line into its fields
    fields = tf.decode_csv(line, FIELD_DEFAULTS)
    
    # Pack the ersult into a dictionary
    features = dict(zip(COLUMNS,fields))
    
    # Separate the label from the features
    label = features.pop('label')
    
    return features, label

### Parse the lines
Datasets有许多个方法在数据被管道到模型的时候处理数据。用得最多得方法是__map__,为Dataset中的每个元素进行转换。

这个__map__方法有一个__map_func__参数来描述怎么样转换每一项(each item)
![Title](map.png)
因此为了解析在csv文件中流出的每一行，将parse_line传递到map方法。示例如下：

经过处理之后，dataset已经不再是包含简单的标量字符串，而包含有(features,labels)对。

剩余的 __iris_data.csv_input_fn__ 函数和 __iris_data.train_input_fn__ 函数一样。

In [ ]:
ds = ds.map(parse_line)
print(ds)

'''
<MapDataset
shapes: (
    {SepalLength: (), PetalWidth: (), ...},
    ()),
types: (
    {SepalLength: tf.float32, PetalWidth: tf.float32, ...},
    tf.int32)>
'''

### Try it out
这个函数用来作为__iris_data.train_input_fn__的替代。可以被用来传递给estimator，示例如下：

Estimators期待收到一个input_fn而不接受其他参数。为了这个限制可以实行，通过使用lambda来捕获参数并且提供期待的交互：

In [ ]:
train_path, test_path = iris_data.maybe_download()

# All the inputs are numeric
feature_columns = [
    tf.feature_column.numeric_column(name)
    for name in iris_data.CSV_COLUMN_NAMES[:-1]
]

# Build the estimator
est = tf.estimator.LinearClassifier(feature_columns,n_classes=3)

# Train the estimator
batch_size = 100
est.train(
    steps=1000,
    input_fn=lambda:iris_data.csv_input_fn(train_path,batch_size)
)

## Summary
__tf.data__ 模块提供了一个类和函数的集合，让从不同的源读取数据变得更容易。更多的是，__tf.data__有简单又有利的方法来提供大部分标准和自定以的转换。

如果说已经有了怎么样将数据有效的载入到Estimator中，考虑以下的文档：
[Creating Custom Estimators](https://www.tensorflow.org/get_started/custom_estimators?hl=zh-cn)展示了怎么样创建自定义的Estimator模型
[Low Level Introduction](https://www.tensorflow.org/programmers_guide/low_level_intro?hl=zh-cn#datasets)表明怎么样用TensorFlow低级的APIs可以操作__tf.data.Dataset__
[importing data](https://www.tensorflow.org/programmers_guide/datasets?hl=zh-cn)介绍Dataset中其他的功能。

In [ ]:
#lambda :i
def odd():
    f=[]
    for i in 'abcdefg':
        f.append((lambda:i))
    return f

#lambda i:i 
def no_odd():
    f=[]
    for i in 'abcdefg':
        f.append((lambda i:i))
    return f

#lambda i=i:i
def f_odd():
    f=[]
    for i in 'abcdefg':
        f.append((lambda i=i:i))
    return f

#lambda a=i:i 
def a_odd():
    f=[]
    for i in 'abcdefg':
        f.append((lambda a=i:i))
    return f


#这是一个简单的for循环
def for_iter(obj):
    for i in obj:
        print (i(),end='')

用 for_iter 循环 odd()，no_odd(), f_odd(),a_odd()  分别会生成‘ggggggg’，error,'abcdefg‘ 和 ‘ggggggg'四种结果
